In [1]:
using Random, Plots
import Statistics
using LinearAlgebra

In [2]:
function refine_walk(walk)
    len = length(walk)
    arr = zeros(2*len-1)
    
    for ind in 1:len-1
        arr[2*ind-1] = walk[ind]
        arr[2*ind] =  (0.5 * (walk[ind] + walk[ind+1]) .+ (((√Δt)/2)*randn(1)))[1] #issue here?
    end
    arr[2*len-1] = walk[len]
    
    return arr
end

refine_walk (generic function with 1 method)

In [3]:
const T = 1
const N = 1000_000
const Δt = T/N

1.0e-6

In [4]:
#Create random walk and wiener process
η = (√Δt) * randn(N)
Wₜ = cumsum(η);

In [5]:
0.5 * (Wₜ[N]^2 + T)

2.9741670165804868

In [6]:
dot(η, Wₜ)

2.9751316873765408

In [7]:
η2 = refine_walk(η); #error here?

In [8]:
Statistics.var(η2),Statistics.var(η)

(8.760255222339793e-7, 1.0019253951834523e-6)

In [9]:
dot(η2, cumsum(η2))

6.755980879154043

# Significantly needs work
for some reason it doesn't seem to converge